<h1>SASSIE Hydrography Buoy Data Use and Visualization</h1>
<p> Hydrography Buoys are drifting platforms that take temperature and salnity measurements at fixed depths as they drift. Hydrobuoy data are downloaded and plotted using the code below! <b>Please run the 'Data Download and Metadata Viewing' and 'Supporting Code' sections in order before running the 'Figure Making Code'.</b> </p>
   <p>The aim of this notebook is to assist the end user in exploratory data analysis by downloading the SASSIE data from NASA's PODAAC, opening the dataset and displaying it's associated metadata, and creating a few visualizations. This notebook was created by Elizabeth Westbrook. For questions and trouble shooting, please email westbrooke@uncw.edu.</p>

# Credential Entry
In this section, you shoud enter your EarthData username and password. <b> DO NOT enter usernames and passwords here which are sensitive.</b> If you do not already have an EarthData account, you can create one <a href="https://urs.earthdata.nasa.gov/">here</a> . </p>
    <p>Please enter your EarthData credentials below: </p>

In [ ]:
#Enter Earthdata Credentials: 
username = 'your_username'
password = 'your_password'

# Data Download and Metadata Viewing

The code in this section will download the dataset from PO.DAAC and open it as an xarray object for metadata and variable attribute viewing.

In [ ]:
import numpy as np
import xarray as xr
import cf_xarray 
import glob 
from datetime import datetime, timedelta 
import matplotlib.pyplot as plt
import cartopy
import matplotlib
import os
import sys
import pandas as pd
import requests 
import cmasher as cmr

<h2>Download The Hydrobuoy data from PO.DAAC</h2>
<p> All data from the SASSIE campaign is stored on NASA's PO.DAAC. The code in this section of the notebook will download SASSIE Hydrobuoy data from PO.DAAC, which is accessed through EarthData. 

In [ ]:
##LOCAL DIRECTORY TO FLOAT DATA
dir_in = 'Data/hydrobuoy/' 

The next block of code creates the directory specified above and downloads the hydrobuoy files to your binder session if it has not already been downloaded.<b> To download the dataset to your local disk from here, right click on the file you want to download and click 'download'. </b>

In [ ]:
##SASSIE DATA DIR
if not os.path.exists(dir_in):
    os.makedirs(dir_in)

buoy_numbers = ['01','02','03','04','05','06','07','08','09','10','11']
for number in buoy_numbers:
    filename = 'SASSIE_Fall_2022_UpTempO_'+number+'_V1.nc'
    if os.path.isfile(dir_in+filename)==False:
        url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_DRIFTER_UPTEMPO_V1/'+filename
        with requests.Session() as session:
                session.auth = (username, password)
                r1 = session.request('get', url)
                r = session.get(r1.url, auth=(username, password))
                if r.status_code == 200:
                    if r.ok:
                        with open(dir_in+filename, 'wb') as f:
                            f.write(r.content) 
                            print('Saving Input File: ' + dir_in+filename) 
                else:
                    print("Error:", r.status_code)
                    if r.status_code == 401:
                        print ('Your Username and/or password are incorrect. Please try again')
    else: 
        print('Buoy '+ number+' file is already in binder directory')

<h2> View The Metadata Inside the Hydrobuoy Float Files</h2>

The netCDF file has global metadata attributes and attributes associated with each variable. This next block will load data and metadata of the netCDF file into an xarray object (ds). <br> The data set will then be displayed in a clickable HTML format. 

In [ ]:
print('Displaying just 1/11 Hydrobuoy files')
#See information about the entire dataset:
files = sorted(glob.glob(dir_in + '/*.nc'))
ds = xr.open_dataset(files[0])
ds

# Supporting Code 
The code in this section provides a set up for the figure making code below by defining directories for data and figures and creating functions that will be called to actually map the data

<h3>Create a Directory to Save Figures</h3>

In [ ]:
#LOCAL DIRECTORY TO SAVE FIGURES
fig_dir ='Figures/hydrobuoy/'
#FIGURE DIR 
if not os.path.exists(fig_dir):
    os.makedirs(fig_dir)

<h3>Define a Colormap and Label for Each Variable in the File</h3>
Within SASSIE's collection of jupyter notebooks, the colormaps used for each variable are held as consistant as possible across all datasets. This function defines the colormap and a label for the variable of interest. 

In [ ]:
#DEFINES COLORMAPS AND LABELS OF EACH VARIABLE IN THIS DATA SET
def define_variable_attributes(var):
    if var =='salinity':
        colormap = 'viridis'
        var_label = 'Salinity'
    if var =='temperature':
        colormap = 'plasma'
        var_label = 'Water Temperature ($^{\circ}$C)'
    return colormap,var_label

<h3>Define a Function to Create a Map of the Study Area</h3>
The following function creates a map of the SASSIE study area, which is defined by minimum and maximum lat/lon values. These ranges can be changed later when the function is called to zoom in/out on the study area.

In [ ]:
def map_study_area(latmin,latmax,lonmin,lonmax):
    
    global fig 
    global ax
    
    #create the map as a figure, set the lat and lon ranges, and add land + river data:
    fig = plt.figure(figsize=(10,8))
    ax = plt.axes(projection=cartopy.crs.NorthPolarStereo(central_longitude=-150))
    ax.set_extent([lonmin,lonmax,latmin,latmax], crs=cartopy.crs.PlateCarree())
    ax.coastlines(color='k')  
    ax.add_feature(cartopy.feature.LAND, facecolor = '0.50',zorder=1)
    ax.add_feature(cartopy.feature.RIVERS,facecolor='blue')
    #Add lat and lon gridlines and labels:
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False, alpha=0.3) #draw_labels=True gives lat labels.
    gl.ylocator = matplotlib.ticker.FixedLocator(np.arange(60,90,1))
    gl.xlocator = matplotlib.ticker.FixedLocator(np.arange(-180,180,2))
    gl.top_labels = False; gl.bottom_labels = True; gl.right_labels = False
    
    #Add markers for reference cities on the coast:
    if (latmin<71.2906) & (lonmin<-156.7886):
        utqiagvik = ax.scatter(-156.7886,71.2906,s=100,transform=cartopy.crs.PlateCarree(),c='red',marker = '*',label='Utqiagvik, AK',zorder=2)
    if (latmin<70.2002) & (lonmax>-148.4597):
        deadhorse = ax.scatter(-148.4597,70.2002,s=100,c='cyan',transform=cartopy.crs.PlateCarree(),marker = '*',label='Deadhorse, AK',zorder=2)
        

<h3>Define a Function to Create a 3D Grid of Latitude, Longitude, and Depth</h3>
The following function creates a 3D grid for plotting 3 dimensional trajectory data from the hydrobuoys

In [ ]:
def make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax):    
        global fig 
        global ax
        
        fig = plt.figure(figsize=(10,8))
        ax = plt.axes(projection='3d',computed_zorder=False)
        ax.xaxis.pane.fill = False; ax.yaxis.pane.fill = False; ax.zaxis.pane.fill = False    
        
        ax.set_xlim(lonmin, lonmax); ax.set_ylim(latmin,latmax); ax.set_zlim(dmin,dmax);
        ax.get_xaxis().get_major_formatter().set_useOffset(False)
        xticks = list(np.linspace(lonmin,lonmax,4))
        ax.set_xticks(np.around(xticks,2))
        ax.set_xticklabels(np.around(xticks,2),rotation=30, ha='center',va='center', minor=False,size='small')

        yticks = list(np.linspace(latmin,latmax,4))
        ax.set_yticks(np.around(yticks,2))
        ax.invert_yaxis()
        ax.set_yticklabels(np.around(yticks,2),rotation=-15,va='bottom', minor=False,size='small')

        zticks = list(np.linspace(dmin,dmax,4))
        ax.set_zticks(np.around(zticks,2))
        ax.set_zticklabels(np.around(zticks,2),ha='center',va='center', minor=False,size='small')

        ax.set_xlabel('Longitude',fontsize=13,labelpad=10,fontweight='bold')
        ax.set_ylabel('Latitude',fontsize=13,labelpad = 7,fontweight='bold')
        ax.set_zlabel('Depth (m)',fontsize=13,labelpad = 16,fontweight='bold')
        ax.view_init(195,280)
        fig.canvas.draw()
        plt.tight_layout()

<h3>Configure Supporting Data to Add to Maps</h3>
<p>The functions for viewing and plotting this data set below have options to include bathymetry and/or shiptrack data to add context to maps. If you are using these options, run
    <br>the following code blocks to:
    <br>1. Create a directory for SASSIE Ship Track data and acess bathymetry data from NOAA
    <br>2. Define functions that add these data to your map when called.</p>

<h4>Create Directory for and Download Shiptrack Data</h4>

In [ ]:
## DIRECTORY TO SHIP TRACK DATA
ship_dir =  'Data/TSG/' 

#DOWLOAD SHIPTRACK DATA
if not os.path.isfile(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc'):
    os.makedirs(ship_dir)
    url = 'https://archive.podaac.earthdata.nasa.gov/podaac-ops-cumulus-protected/SASSIE_L2_SHIPBOARD_TSG_V1/SASSIE_Fall_2022_Shipboard_TSG.nc'
    with requests.Session() as session:
            session.auth = (username, password)
            r1 = session.request('get', url)
            r = session.get(r1.url, auth=(username, password))
            if r.status_code == 200:
                if r.ok:
                    with open(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc', 'wb') as f:
                        f.write(r.content) 
                        print('Saving Input File: ' + ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc') 
            else:
                print("Error:", r.status_code)
                if r.status_code == 401:
                    print ('Your Username and/or password are incorrect. Please try again')
else: 
    print('Shipboard TSG file is already in local directory') 

<h4>Access NOAA Bathymetry Data</h4>

In [ ]:
#READ IN TOPOGRAPHY/BATHYMETRY DATA
url = 'http://ferret.pmel.noaa.gov/thredds/dodsC/data/PMEL/etopo2.nc'
etopodata = xr.open_dataset(url) 

<h4>Define a Function to Index Relevant Bathymetry Data and Add it to the Map</h4>
This function will index bathymetry data from NOAA within the appropriate spatial range and add it to the map. 


In [ ]:
def add_bathy_data(latmin,latmax,lonmin,lonmax):
        topoin = etopodata.rose.values[0:-1:5,1:-1:5]
        lons = etopodata.etopo2_x.values[0:-1:5]
        lats = etopodata.etopo2_y.values[0:-1:5]
        lons_in_range = lons[np.where((lons >lonmin-1) & (lons<lonmax+1))]
        lats_in_range = lats[np.where((lats >latmin-1) & (lats<latmax+1))]
        topo_in_range = np.squeeze(topoin[np.squeeze(np.where((lats >latmin-1) & (lats<latmax+1))),:][:,np.where((lons >lonmin-1) & (lons<lonmax+1))])
        [bathy_lon,bathy_lat] = np.meshgrid(lons_in_range,lats_in_range)
        
        bathy = ax.contour(bathy_lon,bathy_lat,topo_in_range,np.arange(-6000,-1000,300),transform=cartopy.crs.PlateCarree(),cmap='gray',alpha = 0.2,zorder = 0)

<h4>Define a Function to Add Shiptrack Data to a Map</h4>
This function will pull the lat/lon data from the SASSIE Shipboard TSG file and put it onto a map. 

In [ ]:
def add_ship_track():
        ds_ship = xr.open_dataset(ship_dir+'/SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        ship_lat = np.squeeze(ds_ship['latitude'])
        ship_lon = np.squeeze(ds_ship['longitude'])
       
        track = ax.plot(ship_lon, 
                     ship_lat,linewidth = 0.5,
                     c='black',
                       transform=cartopy.crs.PlateCarree(),label = 'Ship Track',zorder=1)

# Figure Making Code 

<h2>Make A Map of Hydrobuoy Tracks</h2>
<p>Use the code in this section to plot the Hydrography Buoy location data on a map colored by time. The code blocks in the 'supporting code' section should be run first.</p>

In [ ]:
##USE THIS TO PLOT THE UPTEMPO TRACKS ON A MAP. ENTER 'all_data' TO MAP ALL UPTEMPO DATA AND 'campaign_data' TO MAP ONLY DATA TAKEN DURRING CAMPIGN.  
def map_uptempo(map_view,ship_track=True,bathymetry_data=True):
    
    ##TIME DATA FROM BOUY 01 (deployed on September 9th and was still collecting data as of 03/13. Since this is the longest running Buoy, we use it to dfine the time range) 
    files = sorted(glob.glob(dir_in + '/*.nc'))
    ds = xr.open_dataset(files[3])
    time = np.squeeze(ds['time'].values)
    start_time = min(time)
    end_time = max(time)
    
    #define the time limits of the campaign:
    campaign_start = datetime(2022,9,8,8)
    campaign_end = datetime(2022,9,29)


    #define the latitue and longitude boundaries of the map:
    if map_view == 'all_data':
        latmin = 70
        latmax =80
        lonmin=-177
        lonmax=-145
        var_min = start_time.astype('int64')
        var_max =end_time.astype('int64')
    if map_view == 'campaign_data':
        latmin = 70
        latmax =74
        lonmin=-157
        lonmax=-145
        ds_ship = xr.open_dataset(ship_dir+'SASSIE_Fall_2022_Shipboard_TSG.nc')
        ship_time = np.squeeze(ds_ship['time'])
        var_min = min(ship_time.values).astype('int64')
        var_max = max(ship_time.values).astype('int64')
    
    #make the map: 
    map_study_area(latmin,latmax,lonmin,lonmax)
    
    #give the map a title:
    if map_view =='all_data':
        ax.set_title('HydroBuoy Tracks',fontsize=20,pad=1) 
    if map_view == 'campaign_data':
        ax.set_title('HydroBuoy Tracks Throughout SASSIE Campaign',fontsize=15,pad=1.5) 
    
    #add optional map add-ons:
    if ship_track==True:
        add_ship_track()
    
    if bathymetry_data == True:    
        add_bathy_data(latmin,latmax,lonmin,lonmax)
    
    ##APPLY THE TIME AND LOCATION DATA IN EACH UPTEMPO FILE 
    files = sorted(glob.glob(dir_in + '/*.nc'))
    if map_view == 'all_data':
        for file in files:
            ds_ut = xr.open_dataset(file)
            ut_time = np.squeeze(ds_ut['time'])
            ut_lat = np.squeeze(ds_ut['latitude'])
            ut_lon = np.squeeze(ds_ut['longitude'])

            deployment_track = ax.scatter(ut_lon,ut_lat,s = 1,
                           c = ut_time,cmap = cmr.neon,
                           transform=cartopy.crs.PlateCarree(),zorder=2,vmin=var_min,vmax=var_max)


        #add a colorbar
        cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
        cbar.set_label(label='Date',size='large',weight='bold')
        cbar_tick_array=(np.linspace(start_time.astype('int64'),end_time.astype('int64'),5))
        cbar.set_ticks(cbar_tick_array)
        cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)  
    if map_view == 'campaign_data':
        for file in files:
            ds_ut = xr.open_dataset(file)
            ds_ut_campaign = ds_ut.sel(time=slice(campaign_start,campaign_end))
            ut_time = np.squeeze(ds_ut_campaign['time'])
            ut_lat = np.squeeze(ds_ut_campaign['latitude'])
            ut_lon = np.squeeze(ds_ut_campaign['longitude'])

            deployment_track = ax.scatter(ut_lon,ut_lat,s = 1,
                           c = ut_time,cmap = cmr.neon,
                           transform=cartopy.crs.PlateCarree(),zorder=2,vmin=var_min,vmax=var_max)


        #add a colorbar
        cbar = fig.colorbar(deployment_track, ax=ax, orientation="horizontal", pad=0.1)
        cbar.set_label(label='Date',size='large',weight='bold')
        cbar_tick_array=(np.linspace(min(ship_time.values).astype('int64'),max(ship_time.values).astype('int64'),5))
        cbar.set_ticks(cbar_tick_array)
        cbar.set_ticklabels(pd.to_datetime(cbar_tick_array).date)  
  
    #SAVE FIGURE
    if not os.path.exists(fig_dir+'maps/'):
        os.makedirs(fig_dir+'maps/')
    print('Saving Output Image:  '+fig_dir+'maps/UpTempO_Buoys'+'_'+map_view+'.png')
    plt.savefig(fig_dir+'maps/UpTempO_Buoys'+'_'+map_view+'.png',dpi='figure',format='png')
    

In [ ]:
map_uptempo('all_data',ship_track=False)   
map_uptempo('campaign_data')   

<h2>Three Dimensional Representation of Sub-Sea Surface Trajectory Data</h2>
Use the code in this section to plot location and depth dataof a selected buoy on a 3D grid, colored by a physical measurement variable from the file (temperature or salinity). The user must select a buoy based on number. The buoy numbers were 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, and 11. 

In [ ]:
def plot_uptempo_data(buoy,var,var_min,var_max):
    
    #open the selected buoy's data set:
    file=files[buoy-1]
    ds = xr.open_dataset(file)
    
    #load the coordinate variables:
    dates = pd.to_datetime(ds.time.values).date
    deployment_date = dates[0]
    lat = np.squeeze(ds['latitude'])
    lon = np.squeeze(ds['longitude'])
    
    if var == 'temperature':
        standard_name = 'sea_water_temperature'
    if var == 'salinity':
        standard_name = 'sea_water_practical_salinity'
    
    #pick out the relevant pyhsical measurement variables:
    var_ds = ds.cf[[standard_name]]
    variables = list(var_ds.keys())
    
    #make an array of depth values:
    depths = np.zeros(len(variables))
    for i in range(len(variables)):
        var_data = ds[variables[i]]
        depths[i] = float(var_data.nominal_depth[0:-1])
        
    ##CREATE THE 3D PLOT
    #define latitude and longitude boundaries. 
    latmin = min(lat.values)
    latmax = max(lat.values)
    lonmin = min(lon.values)
    if (buoy == 4)|(buoy==9): 
        lonmax = -150
    else: 
        lonmax = max(lon.values)
    if ((buoy==2)|(buoy==6)|(buoy==7))&(var=='salinity'):
            dmin = 0
            dmax = 0.5
    else:
        dmin = min(depths)
        dmax = max(depths)
    
    #make the 3D grid: 
    make_3d_grid(latmin,latmax,lonmin,lonmax,dmin,dmax)
    #define a color map and variable label:
    colormap,var_label = define_variable_attributes(var)
    #add a title: 
    ttl = ax.set_title('Buoy'+ ' '+str(buoy)+' '+var_label+' Measurements',fontsize=20,y=0.92,x=0.6)
    
    #APPLY UPTEMPO DATA
    for i in range(len(variables)):
        var_data = ds[variables[i]]
        depth = depths[i]
        a = ax.scatter(lon,lat,depth,c=(var_data.values),cmap = colormap,s=0.5,zorder=1,vmin = var_min,vmax = var_max)
    
    cbar = fig.colorbar(a,shrink=0.6)
    cbar.set_label(label=var_label,size=15,weight='bold',labelpad=10)
   
    ##SAVE FIGURE
    if not os.path.exists(fig_dir+var):
        os.makedirs(fig_dir+var)
    print('Saving Output Image:  '+fig_dir+'/'+var+'/UpTempO_Buoy'+'_'+str(buoy)+'_'+var+'.png')
    plt.savefig(fig_dir+'/'+var+'/UpTempO_Buoy'+'_'+str(buoy)+'_'+var+'.png',dpi='figure',format='png',bbox_inches='tight')

In [ ]:
plot_uptempo_data(3,'temperature',-1.5,1.5)
plot_uptempo_data(3,'salinity',20,30)